In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open('../styles/custom.css', 'r').read()
    return HTML(styles)
css_styling()

# Separation prediction

In this section we will combine the vortex panel method and the boundary layer solver to predict separation on any 2D shape.

## Panel method and boundary layer coupling

What will we need to interface the `VortexPanel` and `BoundaryLayer` modules?  `VortexPanel` doesn't need anything from `BoundaryLayer` - it just needs a geometry and angle of attack.

In [ ]:
import numpy
from matplotlib import pyplot
%matplotlib inline

import VortexPanel as vp
import BoundaryLayer as bl

alpha = numpy.pi/20
foil = vp.make_jukowski(N=64)
vp.solve_gamma_kutta(foil,alpha)
vp.plot_flow(foil,alpha)

From the previous notebook we know the function `march` doesn't need the details of the geometry, but it does need:

- $s$: the distance along the boundary layer
- $u_e$: the velocity on the edge of the boundary layer
- $\nu$: the kinematic viscosity (given by Reynolds number)

We'll need to get the first two from the potential flow solution.

Getting the velocity is deceptively simple. In the first notebook we found that a vortex panel induces a jump in the tangential velocity across itself equal to $-\gamma$. In the second notebook, we applied the no-slip condition on the body side of the panel centers. Therefore the tangential velocity on the flow side of the panel is simply 

$$u_s(x_i,y_i) = -\gamma_i$$

and the external velocity is $u_e=|u_s|$, ie
```python
u_e = abs(vp.get_array(panels,'gamma'))
```
---

Now that we have $u_e$, let's get $s$. Note that a body will form **two** boundary layers, one on each side. We need to identify the starting point of these two flow regions.

##### Quiz 1

Where is the starting point of the two boundary layers?

1. The first panel: `foil[0]`
1. The panel where $u_e = 0$
1. The left-most panel, `foil[N/2]`

---

This makes it straightforward to split the body into the two boundary layer sections:
```python
top = [p for p in panels if p.gamma<=0]    # panels where flow aligns with s
bottom = [p for p in panels if p.gamma>=0] # panels where flow is against s
bottom = bottom[::-1]                      # reverse indices to align bottom
```
Note that we changed the direction of the bottom array so that it runs from the stagnation point to the trailing edge, in accordance with the flow direction.

This is implemented in the `BoundaryLayer.split` function:

In [ ]:
help(bl.split)

We've already set up the foil geometry and solved for $\gamma$, so we can just split it. 

In [ ]:
foil_top,foil_bottom = bl.split(foil)     #3. Split the boundary layers

def plot_segment(panels):
    pyplot.figure(figsize=(10,2))
    pyplot.axis([-1.2,1.2,-.3,.3])
    for i,p_i in enumerate(panels): 
        p_i.plot()
        if i%10 == 0:
            pyplot.scatter(p_i.xc,p_i.yc)
            pyplot.text(p_i.xc,p_i.yc+0.05, 
                'panel ['+'%i'%i+']',fontsize=12)
    pyplot.show()

plot_segment(foil_top)
plot_segment(foil_bottom)

Looks good. 

Now that we've got the boundary layer panels set up, we just use the `VortexPanel.distance` function to get the cumulative distance from the leading edge.

---

The function `BoundaryLayer.panel_march` grabs $u_e$ and $s$ from a boundary layer section of panels and then runs `BoundaryLayer.march`. 

In [ ]:
help(bl.panel_march)

## Boundary layer around a circle

Lets run the example in the help file as our first application: the boundary layer on a circle.

##### Quiz 2

Why do I keep testing code on simple shapes like circles?

1. I'm terribly forgetful
1. New examples take work
1. I want to validate the new code

##### Numerical fundamental: Validation
##### Every piece of code must be tested against a known nontrivial example

---

![Flow around a circle](resources/graphics3.png)
---

So lets validate the results. The *theoretical* laminar separation point for a circular cylinder is $108\deg$ from the leading stagnation point. Since the circle has radius $R=1$, this corresponds to $s=1.88$. Lets check the simulation by plotting $\lambda$. 

__Notice there are 4 steps in solving the BL flow:__

In [ ]:
# set up and solve circle BL flow:
circle = vp.make_circle(N=64)                #1. set-up geometry
vp.solve_gamma_kutta(circle)                 #2. solve flow
top,bottom = bl.split(circle)                #3. split panels
delta,lam,iSep = bl.panel_march(top,nu=1e-5) #4. march along the BL

# plot lambda(s)
s = vp.distance(top)
pyplot.ylabel(r'$\lambda$', fontsize=20)
pyplot.xlabel(r'$s$', fontsize=20)
pyplot.plot(s,lam,lw=2)
pyplot.axhline(-12,ls='--')

As you move along the boundary layer, $\lambda$ decreases from the initial stagnation point condition of $\lambda_0\approx7$ and eventually crosses the threshold for separation $\lambda=-12$. `march` stops once this condition is reached - **therefore the `lam` and `delta` arrays are meaningless after `iSep`**.

From the plot we can see separation occurs between $1.7<s<2$. The `boundaryLayer.sep` functions uses `iSep` to interpolate the value of any array at the separation point:

In [ ]:
sSep = bl.sep(s,iSep)
print('analytic prediction @ s=1.88 \n numeric prediction @ s={:.2f}'.format(sSep))

Our numerical method prediction is less than a percent different than the theoretical result!

We can also use `sep` to plot the separation points together with the flow field:

In [ ]:
nu = 1e-5                                    # set nu
delta,lam,iSep = bl.panel_march(top,nu)      # march along top BL
xSep = bl.sep(vp.get_array(top,'xc'),iSep)   # get x location of separation
ySep = bl.sep(vp.get_array(top,'yc'),iSep)   # get y location of separation 

vp.plot_flow(circle)
pyplot.scatter(xSep, ySep, s=200, c='r')
pyplot.scatter(xSep,-ySep, s=200, c='g')
pyplot.show()

The red and green dots mark the separation point for the top and bottom boundary layer, respectively.

##### Quiz 3

How does the separation point depend on $\nu$?

1. Increasing $\nu$ delays separation
1. Decreasing $\nu$ delays separation
1. Chaning $\nu$ has no effect on separation

---

Is this a bug??

We know from lectures that $\delta$ scales as $\sqrt\nu$, and therefore $\lambda=\frac{\delta^2}\nu u_e'$ doesn't depend on $\nu$ at all. Since the separation point is determined by $\lambda$, this is also independent of $\nu$.

##### Fluids fundamental: Separation Point
##### The point of laminar separation is independent of $Re$

This is **not** true of a turbulent boundary layer.

---

Lets also compare the circle and plate plate boundary layers

In [ ]:
n = int(numpy.floor(iSep)+2) # meaningful BL array lengths

pyplot.ylabel(r'$\delta$', fontsize=20)
pyplot.xlabel(r'$s$', fontsize=20)
pyplot.plot(s,5.836*numpy.sqrt(nu*s),lw=2,label='Flat plate')
pyplot.plot(s[:n],delta[:n],lw=2,label='Circle') # only plot up to separation
pyplot.legend(loc='lower right')

It can't be over-emphasized how different the flat plate and circle boundary layers are. 

##### Fluid fundamental: Pressure gradients
##### Flow gradients completely change the development of a boundary layer.

Because of these gradients:
 - The boundary layer growth is stunted on the front body where the flow is accelerating
 - $\delta$ increases rapidly as the flow approaches the midbody
 - Separation occurs soon after the flow begins to decelerate. 

##### Quiz 4

Up to the point of separation, which geometry will have the larger friction force?

1. The circle
1. The plate  
1. Approximately the same

## Foil at angle of attack

Let's use the functions above to solve for the boundary layer flow over the foil. We will use the same four steps, but the `BoundaryLayer.predict_separation_point` function will march and locate the separation points in the last step:

In [ ]:
alpha = numpy.pi/20
foil = vp.make_jukowski(N=64)    #1. Define the geometry
vp.solve_gamma_kutta(foil,alpha) #2. Solve for the potential flow
foilTop,foilBot = bl.split(foil) #3. Split the boundary layers
xSepTop,ySepTop = bl.predict_separation_point(foilTop) #4a. march over top BL
xSepBot,ySepBot = bl.predict_separation_point(foilBot) #4b. march over bottom BL

vp.plot_flow(foil,alpha)
pyplot.scatter(xSepTop, ySepTop, s=100, c='r')
pyplot.scatter(xSepBot, ySepBot, s=100, c='g')
pyplot.show()

And we see that at an angle of attack, the high pressure side of the foil (the bottom) has delayed separation, while the low pressure side has early separation. This is exactly the kind of information we need to predict stall.

---

Let's review the general solution methodology again: 

1. `make` the geometry as a set of vortex panels
1. `solve` the potential flow by applying the no-slip and kutta conditions
1. `split` the panels into sections on either side of the stagnation points
1. `march` the momentum equation along the boundary layer to the point of separation

Then you can measure, print, or plot any flow quantity you want. 

By combining the `VortexPanel` methods with the `BoundaryLayer.march` method, we've developed a numerical tool to predict separation on **any** body (or group of bodies) defined by a set of panels.

##### Numerical fundamental: Synergy
##### Simple functions can be linked to build powerful methods


## Aspect ratio validation

We've validated the code for the known case of a circle, but this is just one geometry. 

##### Numerical fundamental: Skepticism
##### `But that is what the computer said!' - Famous last words

We should validate a new method on as many known results as possible. 

One thing we could validate is that the separation points change properly if the aspect ratio $t/c$ is changed.  Here is a summary figure from Chapter 3 of Hoerner's *Fluid-Dynamic Drag*

---
![Hoerner Drag, Fig 5, Chap 3](resources/separation_hoerner.png)

---
There are four ellipse examples and two Jukowski examples. Based on this figure, I estimate:

Jukowski foil:

$t/c$| 0.17  | 0.15 
---|
$x/c$| $0.39$  | $0.49$ 

Ellipse:

$t/c$| 1 |1/2 | 1/4 | 1/6 | 1/8 
---|
$x/c$| $0.65$ | $0.75$ | $0.85$ | $0.89$ | $0.92$ 

where $x$ is the linear distance from the nose to the point of separation. I've added the theoretical result for a circle.

##### Your turn #5

Validate the separation prediction method against these two families of geometries. 

 - **Predict** separation for the Jukowski cases, being sure to match $t/c$.
 - **Predict and Plot** $x/c$ for the ellipse for $t/c$=`linspace(0.08,1.1,10)` compared to Hoerner's data. 
 - **Discuss** the results with regards to validation and trends. Are these adequate to validate the numerical method? Are there any surprises?

I recommend that you write a few functions to help automate the prediction process, similar to the previous notebooks...

##### Solution #5


In [ ]:
# your code here

pyplot.scatter([1,1./2.,1./4.,1./6.,1./8.],
               [0.65,0.75,0.85,0.89,0.92], 
               s=100, label='Hoerner')
pyplot.legend(loc='upper right')
pyplot.xlabel(r'$t/c$', fontsize=16)
pyplot.ylabel(r'$x/c$', fontsize=16)